In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
plt.style.use('ggplot')
%matplotlib inline
#load data and set index
universe = pd.read_csv('universe.csv')
df = pd.DataFrame(universe)
df.set_index('ticker', inplace=True)
#fix some columns
df.roa = df.roa.str.replace('%','').astype('float')
df.roe = df.roe.str.replace('%','').astype('float')
df.short_to_float = df.short_to_float.str.replace('%','').astype('float')
df.oper_margin = df.oper_margin.str.replace('%','').astype('float')
df.profit_margin = df.profit_margin.str.replace('%','').astype('float')
df.loc['TSLA','trail_pe'] = 1640
df.trail_pe = df.trail_pe.astype('float')
df.loc['NLOK','px2book'] = 1190
df.px2book = df.px2book.astype('float')
df['ticker2'] = df.index
df['analyst_est'] = ((df['1y_est'] / df['px'])-1)*100
df.perf_1y = df.perf_1y.str.replace('%','').astype('float')

In [7]:
df = df.loc[df['sector']=='Utilities']

In [25]:
#companies with good roe and roa
q=.4
roa = df.loc[df.roa>df.roa.quantile(q),['roa']]
roe = df.loc[df.roe>df.roe.quantile(q),['roe']]

good_ro = pd.merge(roa,roe, how='inner', left_index=True, right_index=True)
good_ro.describe()

,roa,roe
count,11.000000,11.000000
mean,3.623636,207.542727
std,1.450388,651.100311
min,2.780000,9.020000
25%,2.965000,10.295000
50%,3.030000,11.660000
75%,3.480000,12.245000
max,7.830000,2170.680000


In [59]:
#companies with good operating figures
q=.5
curr_ratio = df.loc[df.curr_ratio>df.curr_ratio.quantile(q),['curr_ratio']]
oper_margin = df.loc[df.oper_margin>df.oper_margin.quantile(q),['oper_margin']]
profit_margin = df.loc[df.profit_margin>df.profit_margin.quantile(q), ['profit_margin']]

merge = pd.merge(curr_ratio,oper_margin, how='inner', left_index=True, right_index=True)
good_ops = pd.merge(merge,profit_margin, how='inner', left_index=True, right_index=True)
good_ops.describe()

,curr_ratio,oper_margin,profit_margin
count,5.000000,5.000000,5.000000
mean,1.054000,27.554000,21.850000
std,0.277002,4.653572,8.000128
min,0.840000,23.240000,15.730000
25%,0.870000,24.390000,17.620000
50%,0.900000,25.550000,18.030000
75%,1.170000,30.170000,22.390000
max,1.490000,34.420000,35.480000


In [31]:
#add ons
add_ons = df[['div_yld','_5y_div_yld','analyst_est','name']]
add_ons2 = df[['div_yld','_5y_div_yld','analyst_est','name','pe']]

In [61]:
#companies with reasonable px valuations
q=.65
peg = df.loc[df.peg<df.peg.quantile(q),['peg']]
px2sales = df.loc[df.px2sales<df.px2sales.quantile(q),['px2sales']]
px2book = df.loc[df.px2book<df.px2book.quantile(q),['px2book']]

merge1 = pd.merge(px2book,peg, how='inner', left_index=True, right_index=True)
good_px = pd.merge(merge1,px2sales, how='inner', left_index=True, right_index=True)
good_px.describe()

,px2book,peg,px2sales
count,5.000000,5.000000,5.000000
mean,1.726000,2.958000,2.262000
std,0.285885,0.853622,0.380092
min,1.380000,1.670000,1.790000
25%,1.450000,2.640000,1.940000
50%,1.910000,3.050000,2.420000
75%,1.920000,3.710000,2.470000
max,1.970000,3.720000,2.690000


In [62]:
#companies with reasonable PEs
q=.4
pe = df.loc[df.pe<df.pe.quantile(q),['pe']]
trail_pe = df.loc[df.trail_pe<df.trail_pe.quantile(q),['trail_pe']]
fwd_pe = df.loc[df.fwd_pe<df.fwd_pe.quantile(q),['fwd_pe']]

merge = pd.merge(pe,trail_pe, how='inner', left_index=True, right_index=True)
good_pe = pd.merge(merge,fwd_pe, how='inner', left_index=True, right_index=True)
good_pe.describe()

,pe,trail_pe,fwd_pe
count,6.000000,6.000000,6.000000
mean,12.835000,12.780000,13.575000
std,5.231179,5.244525,3.061625
min,2.620000,2.630000,8.400000
25%,13.677500,13.512500,12.200000
50%,13.800000,13.845000,14.645000
75%,15.122500,14.695000,15.747500
max,17.590000,17.980000,16.260000


In [65]:
#companies with good ev ratios
q=.9
ev2rev = df.loc[df.ev2rev<df.px2book.quantile(q), ['ev2rev']]
ev2ebitda = df.loc[df.ev2ebitda<df.ev2ebitda.quantile(q),['ev2ebitda']]

good_ev = pd.merge(ev2rev, ev2ebitda, how='inner', left_index=True, right_index=True)
good_ev.describe()

,ev2rev,ev2ebitda
count,10.000000,10.000000
mean,3.488000,11.547000
std,0.848119,3.187667
min,1.690000,7.940000
25%,3.402500,9.295000
50%,3.655000,10.855000
75%,3.917500,14.215000
max,4.460000,17.060000


In [66]:
# good_ops_ro
good_ops_ro = pd.merge(good_ops, good_ro, how='inner', left_index=True, right_index=True)
pd.merge(good_ops_ro, add_ons2, how='inner', left_index=True, right_index=True)

,curr_ratio,oper_margin,profit_margin,roa,roe,div_yld,_5y_div_yld,analyst_est,name,pe
ticker,,,,,,,,,,
AWK,0.87,34.42,17.62,3.47,10.42,1.33,1.78,0.103137,"American Water Works Company, Inc.",45.16
ATO,1.49,30.17,22.39,3.49,9.60,2.72,2.15,14.114675,Atmos Energy Corporation,17.69


In [67]:
good_pe_px = pd.merge(good_pe, good_px, how='inner', left_index=True, right_index=True)
good_pe_px_ev = pd.merge(good_pe_px, good_ev, how='inner', left_index=True, right_index=True)
good_pe_px_ev = pd.merge(good_pe_px_ev, add_ons, how='inner', left_index=True, right_index=True)
good_pe_px_ev

,pe,trail_pe,fwd_pe,px2book,peg,px2sales,ev2rev,ev2ebitda,div_yld,_5y_div_yld,analyst_est,name
ticker,,,,,,,,,,,,
PNW,13.66,13.41,14.9,1.45,3.72,2.42,4.3,9.64,4.33,3.38,14.088182,Pinnacle West Capital Corporation


In [55]:
q=.5
good_esg = df.loc[df.esg_score<df.esg_score.quantile(q),['esg_score']]
good_esg.describe()

,esg_score
count,13.000000
mean,27.769231
std,2.241794
min,23.000000
25%,28.000000
50%,29.000000
75%,29.000000
max,30.000000


In [52]:
q=.6
board = df.loc[df.board<df.board.quantile(q),['board']]
sh_holdr_rgts = df.loc[df.sh_holdr_rgts<df.sh_holdr_rgts.quantile(q),['sh_holdr_rgts']]
comp = df.loc[df.comp<df.comp.quantile(q),['comp']]
merge = pd.merge(sh_holdr_rgts,board, how='inner', left_index=True, right_index=True)
good_gov = pd.merge(merge,comp, how='inner', left_index=True, right_index=True)
good_gov.describe()

,sh_holdr_rgts,board,comp
count,8.000000,8.000000,8.00000
mean,2.875000,3.000000,2.25000
std,1.356203,1.772811,1.38873
min,1.000000,1.000000,1.00000
25%,2.500000,1.000000,1.00000
50%,3.000000,3.500000,2.00000
75%,3.250000,4.250000,3.25000
max,5.000000,5.000000,4.00000


In [56]:
gov = pd.merge(good_gov,good_esg, how='inner', left_index=True, right_index=True)
pd.merge(gov,add_ons2, how='inner', left_index=True, right_index=True)


,sh_holdr_rgts,board,comp,esg_score,div_yld,_5y_div_yld,analyst_est,name,pe
ticker,,,,,,,,,
EIX,1.0,1.0,3.0,29.0,4.54,3.44,21.357559,Edison International,60.64
EXC,3.0,5.0,1.0,24.0,3.58,3.42,13.476837,Exelon Corporation,17.59


In [70]:
#most bullish analyst estimates:
bull = df['analyst_est'].sort_values(ascending=False).head(5)
add_ons = df[['name','sector','div_yld','pe','beta']]
pd.merge(bull, add_ons, how='left',left_index=True, right_index=True)\
.sort_values(by='analyst_est', ascending=False)

,analyst_est,name,sector,div_yld,pe,beta
ticker,,,,,,
EIX,21.357559,Edison International,Utilities,4.54,60.64,0.61
ETR,19.600592,Entergy Corporation,Utilities,4.02,13.73,0.53
AEP,18.784041,"American Electric Power Company, Inc.",Utilities,3.75,20.44,0.25
AEE,17.681918,Ameren Corporation,Utilities,2.84,21.20,0.25
LNT,17.244919,Alliant Energy Corporation,Utilities,3.31,18.13,0.33


In [68]:
#most bearish analyst estimates:
bear = df['analyst_est'].sort_values(ascending=True).head(5)
add_ons = df[['name','sector','div_yld','pe','beta']]
pd.merge(bear, add_ons, how='left',left_index=True, right_index=True)\
.sort_values(by='analyst_est', ascending=True)

,analyst_est,name,sector,div_yld,pe,beta
ticker,,,,,,
AWK,0.103137,"American Water Works Company, Inc.",Utilities,1.33,45.16,0.22
AES,1.035714,The AES Corporation,Utilities,2.15,NaN,0.87
ED,2.339181,"Consolidated Edison, Inc.",Utilities,4.32,17.72,0.16
SO,3.915810,The Southern Company,Utilities,4.18,20.51,0.43
NEE,6.432400,"NextEra Energy, Inc.",Utilities,1.67,56.72,0.19


In [69]:
q=.80
bear = df['analyst_est'].sort_values(ascending=True).head(25)
short_ratio = df.loc[df.short_ratio>df.short_ratio.quantile(q),['short_ratio']]
short_to_float = df.loc[df.short_to_float>df.short_to_float.quantile(q),['short_to_float']]
add_ons = df[['name','sector','div_yld','pe','beta']]

short = pd.merge(short_ratio,short_to_float, how='inner', left_index=True, right_index=True)
short_bear = pd.merge(short,bear, how='inner', left_index=True, right_index=True)
short_bear = pd.merge(short_bear,add_ons, how='inner', left_index=True, right_index=True)
short_bear

,short_ratio,short_to_float,analyst_est,name,sector,div_yld,pe,beta
ticker,,,,,,,,
CNP,4.06,3.47,15.152941,"CenterPoint Energy, Inc.",Utilities,3.01,NaN,0.98
FE,3.68,4.38,15.038560,FirstEnergy Corp.,Utilities,5.01,23.21,0.22
PNW,3.44,3.54,14.088182,Pinnacle West Capital Corporation,Utilities,4.33,13.66,0.29
